In [1]:
import os
import time


import numpy as np

# import matplotlib
# import matplotlib.pyplot as plt
# from matplotlib.gridspec import GridSpec

from preferentialBO.test_functions import test_functions
from preferentialBO.src.preferential_bayesian_opt import *

In [2]:
(func_name, BO_method, initial_seed) = "Branin", "HB-EI", 0

test_func = eval("test_functions." + func_name)()

x_bounds = test_func.bounds
input_dim = test_func.d
interval_size = x_bounds[1] - x_bounds[0]
kernel_bounds = np.array([interval_size * 1e-1, interval_size / 2.0])
X_all = None

# seed for reproducibility
np.random.seed(initial_seed)

# First setting
num_duels = 3 * input_dim
noise_std = 1e-2

X_init = (
    np.random.rand(2 * num_duels, input_dim) * (x_bounds[1] - x_bounds[0]) + x_bounds[0]
)
y_init = test_func.values(X_init)

X_init = X_init.reshape(num_duels, 2, input_dim)
y_init = y_init.reshape(num_duels, 2, 1)
sort_idx = np.argsort(y_init, axis=1)

# sort X so that first d dimension is a winner and remained d dimension is a looser
X_init_sort = np.take_along_axis(X_init, sort_idx, axis=1)[:, ::-1, :].reshape(
    num_duels, 2 * input_dim
)

kernel = GPy.kern.RBF(
    input_dim=input_dim,
    lengthscale=0.5 * (kernel_bounds[1] - kernel_bounds[0]) + kernel_bounds[0],
    variance=1,
    ARD=True,
)

# Brochu et al., 2010
if BO_method == "LA-EI":
    model_name = "Laplace"
    first_acq = "max_mean"
    second_acq = "EI"
# Sui et al., 2017
elif BO_method == "EP-KSS":
    model_name = "EP"
    first_acq = "TS"
    second_acq = "TS"
elif BO_method == "EP-EI":
    model_name = "EP"
    first_acq = "max_mean"
    second_acq = "EI"
elif BO_method == "EP-MUC":
    model_name = "EP"
    first_acq = "max_mean"
    second_acq = "MUC"
# proposed
elif BO_method == "HB-EI":
    model_name = "Gibbs"
    first_acq = "current_best"
    second_acq = "EI"
# proposed
elif BO_method == "HB-UCB":
    model_name = "Gibbs"
    first_acq = "current_best"
    second_acq = "UCB"
# Benavoli et al., 2020
elif BO_method == "DuelTS":
    model_name = "Gibbs"
    first_acq = "current_best"
    second_acq = "TS"
# Benavoli et al., 2020, 2021
elif BO_method == "DuelUCB":
    model_name = "Gibbs"
    first_acq = "current_best"
    second_acq = "UCB"
# Benavoli et al., 2020, 2021
elif BO_method == "EIIG":
    model_name = "Gibbs"
    first_acq = "current_best"
    second_acq = "EIIG"
else:
    print("Specified method is not implemented")
    exit()

# bayesian optimizer
if model_name in ["Laplace", "EP"]:
    optimizer = PreferentialBO_GaussApprox(
        X_init_sort, x_bounds, kernel, kernel_bounds, noise_std, GPmodel=model_name
    )
elif "DuelUCB" in BO_method or "EIIG" in BO_method or "MCMC" in BO_method:
    optimizer = PreferentialBO_MCMC(
        X_init_sort, x_bounds, kernel, kernel_bounds, noise_std, GPmodel=model_name
    )
elif "Gibbs" == model_name:
    optimizer = PreferentialBO_HallucinationBeliever(
        X_init_sort, x_bounds, kernel, kernel_bounds, noise_std, GPmodel=model_name
    )

if "MCMC" in BO_method:
    optimizer.GPmodel.thinning = 10
    optimizer.GPmodel.sample_size = 10000

Regret_list = list()
computation_time_list = [0]

ITR_MAX = 110
for i in range(ITR_MAX):
    print("-------------------------------------")
    print(str(i) + "th iteration")
    print("-------------------------------------")

    # compute next input
    start = time.time()
    if X_all is None:
        x1, x2 = optimizer.next_input(first_acq=first_acq, second_acq=second_acq)
    else:
        x1, x2 = optimizer.next_input_pool(
            first_acq=first_acq, second_acq=second_acq, X=X_all
        )

    print("Duel is {} {}".format(x1, x2))

    # evaluate preference (The black-box duel is performed here, but we use new_output only for duels.)
    new_output = test_func.values(np.r_[x1, x2])
    if new_output[0] > new_output[1]:
        optimizer.update(X_win=x1, X_loo=x2)
    else:
        optimizer.update(X_win=x2, X_loo=x1)

    # tuning hyperparameters for per 10 iterations
    if (i + 1) % 10 == 0:
        optimizer.GPmodel.model_selection()

    computational_time = time.time() - start

    # compute recommendation point (without time measurement)
    if first_acq in ["current_best", "max_mean"]:
        inference_point = x1
    else:
        print("not implemented first acq")
        exit()

    Regret_list.append(test_func.values(np.atleast_2d(inference_point)).ravel())

    # add computational time
    computation_time_list.append(computational_time)

    # output intermediate result
    if i % 5 == 0:
        print(
            "time {}, function value at recommendation point {}".format(
                np.sum(computation_time_list), Regret_list[-1]
            )
        )

-------------------------------------
0th iteration
-------------------------------------


 /Users/takeno/miniconda3/lib/python3.7/site-packages/scipy/stats/_qmc.py:1078: UserWarning:The balance properties of Sobol' points require n to be a power of 2.


optimized acquisition function value: [0.38722082]
Duel is [[-4.69672404 12.48929768]] [[5.1908257  4.34071331]]
time 1.2289550304412842, function value at recommendation point [-24.56049543]
-------------------------------------
1th iteration
-------------------------------------
optimized acquisition function value: [0.37024054]
Duel is [[5.1908257  4.34071331]] [[7.10006751 1.79847857]]
-------------------------------------
2th iteration
-------------------------------------
optimized acquisition function value: [0.46667986]
Duel is [[7.10006751 1.79847857]] [[10.          3.13259345]]
-------------------------------------
3th iteration
-------------------------------------
optimized acquisition function value: [0.22049817]
Duel is [[10.          3.13259345]] [[10.          3.78702753]]
-------------------------------------
4th iteration
-------------------------------------
optimized acquisition function value: [0.27746213]
Duel is [[10.          3.13259345]] [[7.94586791 3.2848991

KeyboardInterrupt: 